---

# **[실습] 주택청약 FAQ 시스템 구현**

### **문제 설명**
이전 코드를 기반으로 주택청약 FAQ 시스템을 다음 요구사항에 맞춰 개선합니다. 

1. 응답 품질 향상 (1개 이상)
   - 생성된 답변의 품질을 평가 (답변이 불충분한 경우 예외 처리)
   - 관련성 높은 FAQ 문서 검색 (임베딩 모델, 청크 크기, 벡터 검색 방법 등)

2. 사용자 경험 개선 (1개 이상)
   - 대화 이력 관리 기능 추가 (요약, 트리밍 기능 등 고려)
   - 최근 대화 기반 컨텍스트 구성 
   - 사용자 프로필 기반 맞춤 응답

### **제약 조건**
- Gradio ChatInterface 사용
- RAG 구조 유지

In [ ]:
## 문제 해결 방향 
### 응답  품질 향상 
#### 1) 임베딩 모델 변경 
- 한국어 사용에 강점이 있는 모델로 변경 

#### 2) 벡터 검색 방법 변경 
- mmr 파라미터 조절 및 메타데이터 필터링 적용

### 사용자 경험 개선 

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.9, 
)

# 문서 로드

- `TextLoader`를 이용한 텍스트 문서 로드

In [5]:
from langchain_community.document_loaders import TextLoader

### 파일 경로 설정
faq_text_file = "data/housing_faq.txt"

### 파일 로더 초기화 
txt_loader = TextLoader(faq_text_file, encoding="utf-8")

### 로딩 
docs = txt_loader.load()

# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


- 문서 전처리

In [6]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [7]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


- QA 쌍을 문자열 포맷팅 후 문서 객체로 변환 

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 텍스트 분석 전문가입니다. 
주어진 텍스트에서 중요한 키워드를 추출하고, 텍스트의 간단한 요약을 작성하는 것이 당신의 역할입니다.

## 추출 지침:
- 텍스트의 맥락을 고려하여 핵심 용어나 전문 용어를 추출합니다
- 주요 아이디어나 원리, 개념을 포함합니다
- 가장 중요한 키워드를 1개 추출합니다
- 요약은 1문장으로 간결하게 작성합니다

## 출력 형식:
- keyword: 가장 중요한 키워드 
- summary: 텍스트의 간단한 요약"""),
    
    ("user", "다음 텍스트를 분석해주세요:\n\n{input_text}")
])

# LCEL 체인 구성
llm_with_structure = llm.with_structured_output(KeywordOutput) 
keyword_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyword_extractor.invoke(qa_pairs[0]['question']+qa_pairs[0]['answer'])
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 해당 주택건설지역
요약: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.


In [10]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # QA 쌍을 포맷팅
        formatted_output = (
            f"[{pair['number']}]\n"
            f"질문: {pair['question']}\n"
            f"답변: {pair['answer']}\n"
        )

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=formatted_output,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1,
 'summary': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.'}


- 문서 객체 포맷팅 구성

In [11]:
from langchain_core.documents import Document

def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
            }
        )
        processed_docs.append(doc)

    return processed_docs


In [12]:
# QA 쌍 포맷팅
summary_formatted_docs = format_qa_pairs_with_summary(qa_pairs) 
print(f"포맷팅된 문서 개수: {len(summary_formatted_docs)}")

# 문서 확인
print(summary_formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1}


## 벡터 저장
- OpenAI 모델에서 허깅페이스로 변경 
  - AS-IS : text-embedding-3-small
  - TO-BE : jhgan/ko-sroberta-multitask
  
### 모델 선정 이유 
- 한국어 처리에 강점이 있는 모델 

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# 문서 벡터 저장
vector_store = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="enhance_housing_faq_db",
    persist_directory="./chroma_db",
)

In [16]:
output_file = "data/housing_faq_formatted.json"

with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

포맷팅된 문서를 data/housing_faq_formatted.json에 저장했습니다.


-  벡터 저장소 로드

In [19]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="enhance_housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

In [20]:
print(f"벡터 저장소에 저장된 문서 개수: {vector_store._collection.count()}")

벡터 저장소에 저장된 문서 개수: 50


### 검색기 생성 

- search_kwargs 변경 
  - 검색 결과간 다양성을 위해 `fetch_k` 파라미터 증가 

In [27]:
# 테스트 질문
target_keyword = "청약통장"
query = "청약통장에 대한 정보를 찾아주세요"

In [29]:
mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3, 
        "fetch_k": 10, 
        "lambda_mult": 0.7,
        "filter":{"keyword":{"$eq": target_keyword}} 
        }
)

In [30]:
### mmr 검색 테스트
results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

[43]
질문: 청약통장이 압류 등으로 사용에 제한이 있거나, 청약통장 담보대출을 받고 있는 경우에도 해당 통장을 이용하여 당첨이 가능한지?
답변: 압류 등의 상태라고 하더라도 청약통장이 해지되지 않고 유지 중이라면 청약신청이 가능합니다. 청약통장 담보대출을 받은 경우에도 청약신청이 가능합니다. 다만, 담보대출 연체 등으로 해당 은행에서 청약통장을 해지하고 담보대출 상환처리하는 경우 새로 청약통장을 가입하여 청약 신청해야 하는 점 유의하시길 바랍니다.

--------------------------------------------------
청약통장
43
청약통장이 압류 등으로 사용에 제한이 있거나, 청약통장 담보대출을 받고 있는 경우에도 해당 통장을 이용하여 당첨이 가능한지?
압류 등의 상태라고 하더라도 청약통장이 해지되지 않고 유지 중이라면 청약신청이 가능합니다. 청약통장 담보대출을 받은 경우에도 청약신청이 가능합니다. 다만, 담보대출 연체 등으로 해당 은행에서 청약통장을 해지하고 담보대출 상환처리하는 경우 새로 청약통장을 가입하여 청약 신청해야 하는 점 유의하시길 바랍니다.


### RAG Chain 구성 
- 참조된 문서를 기반으로 답변

In [42]:
from langchain_core.prompts import ChatPromptTemplate

template ="""Answer all the question in Korean(한국어). 
당신은 주택청약 전문가입니다. 
제공되는 '참고 문서'의 내용만을 사용하여 사용자의 질문에 답변해야 합니다.

[참고 문서]
{context}

[질문]
{question}

[답변]
"""

prompt = ChatPromptTemplate.from_template(template)


- 문서 포맷팅 함수 

In [43]:
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

- 작동 확인 테스트

In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chain 구성
test_retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

rag_chain = (
    {'context': test_retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "청약 통장에 대해서 알려주세요"
rag_chain.invoke(query)

'청약 통장에 대해 설명드리겠습니다.\n\n청약 통장은 주택을 분양받기 위해 가입하는 저축통장으로, 종류에 따라 국민주택, 민영주택 등 청약 가능한 주택 유형과 조건이 다릅니다.\n\n1. **주택청약종합저축**  \n- 국민주택과 민영주택 모두 청약할 수 있는 통장입니다.  \n- 기존 청년우대형 주택청약종합저축 가입자는 별도 신청 없이 청년주택드림청약통장으로 자동 전환됩니다.  \n- 일반 주택청약종합저축 가입자는 은행 방문 후 전환 신청이 필요하며, 당첨계좌인 경우에는 재가입해야 합니다.\n\n2. **청약저축**  \n- 2003년 이전에는 국민주택 청약을 위한 저축이었으며, 민영주택 청약은 불가능했습니다.  \n- 다만, 청약저축 가입자가 민영주택에 청약하려면 청약예금으로 1회에 한해 전환할 수 있습니다.  \n- 전환 조건은 청약저축 납입인정금액이 민영주택 청약 예치기준금액 이상이어야 하며, 전환은 입주자모집공고일 전일까지 은행에서 완료해야 합니다.  \n- 전환 후에는 청약저축으로 재전환할 수 없습니다.\n\n3. **청약예금 및 청약부금**  \n- 민영주택 청약 시 입주자모집공고일 기준으로 해당 주택 전용면적에 맞는 예치기준 금액 이상이 납입되어 있어야 청약이 인정됩니다.  \n- 청약예금 가입자는 입주자모집공고일 전일까지 주택면적 변경이 가능하며, 변경 후 면적으로 청약 신청할 수 있습니다.  \n- 청약부금 가입자가 85㎡ 초과 민영주택에 청약하려면 입주자모집공고일 전일까지 청약예금으로 전환해야 합니다.\n\n4. **기타**  \n- 청약저축에서 민영주택 청약예금으로 전환은 은행 영업점 방문이 필수이며, 인터넷 신청은 불가능합니다.  \n- 통장 예치금은 입주자모집공고일 기준으로 납입되어 있어야 인정됩니다.\n\n요약하면, 청약 통장은 주택 유형과 가입 시기에 따라 청약 가능한 주택과 전환 조건이 다르므로, 청약하려는 주택 유형과 시기에 맞게 통장 종류와 납입금액을 확인하고 관리하는 것이 중요합니다.'

- 문서와 포맷팅된 컨텍스트를 반환하는 함수

In [50]:
from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """
    docs = mmr_retriever.invoke(question)
    return {
        "question": question,  # 질문
        "context": format_docs(docs),   # 문서 포맷팅된 컨텍스트
        "source_documents": docs   # 검색된 문서 리스트
    }

- 컨텍스트와 질문을 받아 답변 생성

In [51]:
def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = prompt | llm | StrOutputParser()
    answer = answer_chain.invoke(input_data)

    return {
        "answer": answer,  # 생성된 답변 (answer_chain 결과)
        "source_documents": input_data["source_documents"]  # 소스 문서 정보 (input_data에서 가져옴)
    }

- RAG 체인 구성

In [52]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    RunnableLambda(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': RunnableLambda(prompt_and_generate_answer),  # 답변 생성
        'question': itemgetter("question"),
        "source_documents": itemgetter("source_documents")  # 소스 반환
    }
)

In [53]:
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

답변: [답변]  
참고 문서에 수원시의 주택건설지역에 관한 내용이 포함되어 있지 않아 정확한 답변을 드리기 어렵습니다. 추가적인 자료나 해당 지자체의 공식 정보를 참고하시길 권장합니다.

참조 문서:

문서 1:
내용: [43]
질문: 청약통장이 압류 등으로 사용에 제한이 있거나, 청약통장 담보대출을 받고 있는 경우에도 해당 통장을 이용하여 당첨이 가능한지?
답변: 압류 등의 상태라고 하더라도 청약통장이 해지되지 않고 유지 중이라면 청약신청이 가능합니다. 청약통장 담보대출을 받은 경우에도 청약신청이 가능합니다. 다만, 담보대출 연체 등으로 해당 은행에서 청약통장을 해지하고 담보대출 상환처리하는 경우 새로 청약통장을 가입하여 청약 신청해야 하는 점 유의하시길 바랍니다.



In [54]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")


문서 1:
내용: [43]
질문: 청약통장이 압류 등으로 사용에 제한이 있거나, 청약통장 담보대출을 받고 있는 경우에도 해당 통장을 이용하여 당첨이 가능한지?
답변: 압류 등의 상태라고 하더라도 청약통장이 해지되지 않고 유지 중이라면 청약신청이 가능합니다. 청약통장 담보대출을 받은 경우에도 청약신청이 가능합니다. 다만, 담보대출 연체 등으로 해당 은행에서 청약통장을 해지하고 담보대출 상환처리하는 경우 새로 청약통장을 가입하여 청약 신청해야 하는 점 유의하시길 바랍니다.

평가 결과: no
